<a href="https://colab.research.google.com/github/user77772772/models/blob/main/CIFAR_10_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])


In [3]:
train_set=datasets.CIFAR10(root='data',train=True, download=True, transform=transform)
test_set=datasets.CIFAR10(root='data',train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:03<00:00, 48.1MB/s]


In [5]:
train_loader=DataLoader(train_set,batch_size=32,shuffle=True, num_workers=2)
test_loader=DataLoader(test_set,batch_size=32,shuffle=False,num_workers=2)

In [8]:
# Take look at data
image,label= train_set[0]

In [7]:
image.size()

torch.Size([3, 32, 32])

In [9]:
class_names=['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)   # (3 → 16 filters)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)  # (16 → 32 filters)

        # After two convs + two pools: (32, 8, 8) → flatten = 2048
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)

        X = X.view(-1, 32 * 8 * 8)   # flatten
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=1)


In [15]:
torch.manual_seed(42)
model=CNNModel()
model

CNNModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [16]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [17]:
epochs=5

In [20]:
epochs = 5
train_loss = []
test_loss = []
train_correct = []
test_correct = []

# For loop of epochs
for epoch in range(epochs):
  trn_corr = 0
  tst_corr= 0

# Train
  for b,(x_train, y_train) in enumerate(train_loader):
    b += 1 # Start our batches at 1
    y_pred = model(x_train)
    loss = criterion(y_pred,y_train)
    predicted = torch.max(y_pred ,1)[1]
    batch_corr = (predicted==y_train).sum()
    trn_corr += batch_corr

    # Update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print some results
    if b%600 == 0:
      print(f'Epoch: {epoch},batch: {b} Loss: {loss.item()}')

  train_loss.append(loss)
  train_correct.append(trn_corr)

# Test
  with torch.no_grad():
    for b,(x_test, y_test) in enumerate(test_loader):
      y_val = model(x_test)
      predicted = torch.max(y_val, 1)[1]
      tst_corr += (predicted == y_test).sum()

  loss = criterion(y_val ,y_test)
  test_loss.append(loss)
  test_correct.append(tst_corr)

Epoch: 0,batch: 600 Loss: 1.5089830160140991
Epoch: 0,batch: 1200 Loss: 1.6608132123947144
Epoch: 1,batch: 600 Loss: 1.6788996458053589
Epoch: 1,batch: 1200 Loss: 1.2140915393829346
Epoch: 2,batch: 600 Loss: 1.2247045040130615
Epoch: 2,batch: 1200 Loss: 1.3077939748764038
Epoch: 3,batch: 600 Loss: 1.1974835395812988
Epoch: 3,batch: 1200 Loss: 1.1432147026062012
Epoch: 4,batch: 600 Loss: 1.0317317247390747
Epoch: 4,batch: 1200 Loss: 1.0991932153701782


In [23]:
correct=0
total=0
model.eval()
with torch.no_grad():
  for images,label in test_loader:
    outputs=model(images)
    predicted=torch.max(outputs,1)[1]
    total+=label.shape[0]
    correct+=(predicted==label).sum().item()

accuracy=correct/total


In [24]:
accuracy

0.6107

In [35]:

# Define transform: Resize first, then ToTensor, then Normalize
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),   # ✅ resize while still a PIL image
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

def load_image(image_path):
  image=Image.open(image_path).convert("RGB")
  image=new_transform(image)
  image=image.unsqueeze(0)
  return image

image_path=['dog.jpg','truck.jpg']
images=[load_image(img) for img in image_path]
model.eval()
with torch.no_grad():
  for image in images:
    output=model(image)
    predicted=torch.max(output,1)[1]
    print(f'prediction: {class_names[predicted.item()]}')



prediction: dog
prediction: truck
